## 0. Setup

In [1]:
import os
from typing import Dict, Any, List
from IPython.display import Image, display
import nest_asyncio
from dotenv import load_dotenv
from urllib.parse import urlparse
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from tavily import TavilyClient

In [2]:
nest_asyncio.apply()

In [3]:
load_dotenv()

True

In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("VOCAREUM_API_KEY")
)

## 1. Intake Agent

In [5]:
@tool
def outline_topic(topic: str) -> Dict[str, Any]:
    """
    Creates a pragmatic outline for a blog post given a topic.

    Args:
        topic (str): The domain/area/technology (e.g., "LangGraph multi-agent systems").

    Returns:
        Dict[str, Any]: A dictionary with:
            - title (str): Suggested working title.
            - audience (str): Target reader profile.
            - sections (List[str]): Ordered list of section headings.

    Example:
        >>> outline_topic(topic="LangGraph multi-agent systems")
        {'title': 'LangGraph multi-agent systems: What It Is, Why It Matters, and How To Start',
         'audience': 'Practitioners and curious newcomers',
         'sections': ['Introduction', 'Why ... matters right now', ...]}
    """
    base = topic.strip().rstrip(".")
    title = f"{base}: What It Is, Why It Matters, and How To Start"
    audience = "Practitioners and curious newcomers"
    sections = [
        "Introduction",
        f"Why {base} matters right now",
        f"Core concepts in {base}",
        f"Practical getting-started guide for {base}",
        "Common pitfalls and best practices",
        f"Trends and the future of {base}",
        "Conclusion and next steps",
    ]
    return {"title": title, "audience": audience, "sections": sections}

In [6]:
@tool
def generate_keywords(topic: str, limit: int = 12) -> List[str]:
    """
    Generates a compact keyword set for SEO checks based on a topic.

    Args:
        topic (str): The domain/area/technology to derive keywords from.
        limit (int, optional): Maximum number of keywords to return. Defaults to 12.

    Returns:
        List[str]: A list of keyword phrases ordered by relevance heuristics.

    Example:
        >>> generate_keywords(topic="LangGraph multi-agent systems", limit=8)
        ['langgraph multi-agent systems', 'langgraph multi-agent systems tutorial', ...]
    """
    root = topic.lower().strip()
    extras = [
        f"{root} tutorial", f"{root} guide", f"{root} best practices",
        f"{root} examples", f"{root} use cases", f"{root} vs alternatives",
        f"{root} performance", f"{root} scalability",
        f"{root} architecture", f"{root} tools", f"{root} pitfalls"
    ]
    # Deduplicate while preserving order
    seen = set()
    out = []
    for k in [root] + extras:
        if k not in seen:
            seen.add(k)
            out.append(k)
        if len(out) >= limit: break
    return out

In [7]:
intake_agent = create_react_agent(
    name="intake_agent",
    prompt=SystemMessage(
        content=(
            "You are a pragmatic content planner. "
            "Given a domain/area/technology topic, produce a blog plan: "
            "a clear working title, target audience, and a section outline. "
            "Use tools to create the outline and initial keyword set. "
            "Return ONLY JSON with keys: topic, title, audience, sections, keywords."
        )
    ),
    model=llm,
    tools=[outline_topic, generate_keywords],
)

In [8]:
user_trigger_message = HumanMessage(content="I want a blog post about MCP")

In [9]:
intake_agent_result = intake_agent.invoke(
    input={
        "messages": [
            user_trigger_message
        ]
    }
)

In [10]:
for message in intake_agent_result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I want a blog post about MCP
================================== Ai Message ==================================
Name: intake_agent
Tool Calls:
  outline_topic (call_ydui18iGD7nAfxzxwY6KxAhS)
 Call ID: call_ydui18iGD7nAfxzxwY6KxAhS
  Args:
    topic: MCP
  generate_keywords (call_STJ4VXPABFIOeGyvxPRTBmRV)
 Call ID: call_STJ4VXPABFIOeGyvxPRTBmRV
  Args:
    topic: MCP
================================= Tool Message =================================
Name: outline_topic

{"title": "MCP: What It Is, Why It Matters, and How To Start", "audience": "Practitioners and curious newcomers", "sections": ["Introduction", "Why MCP matters right now", "Core concepts in MCP", "Practical getting-started guide for MCP", "Common pitfalls and best practices", "Trends and the future of MCP", "Conclusion and next steps"]}
================================= Tool Message =================================
Name: generate_keywords

["mc

## 2. Researcher Agent

In [11]:
tavily_client = TavilyClient(
    api_key=os.getenv("TAVILY_API_KEY")
)

In [12]:
@tool
def web_search(question:str)->Dict[str, Any]:
    """
    Performs a web search and returns top search results for a given query.

    Args:
        question (str): A query to be searched.

    Returns:
        Dict[str, Any]: A dictionary with a 'results' list where each item is a dict containing:
            - url (str): The result URL.
            - title (str): The page title.
            - content (str): A content snippet or summary.
            - score (float): Relevance score from the search provider.

    Example:
        >>> web_search(question="LangGraph multi-agent patterns")
        {'results': [{'url': 'https://...', 'title': '...', 'content': '...', 'score': 0.82}, ...]}
    """
    response = tavily_client.search(
        query=question,
        max_results=5,
    )
    return response

In [13]:
@tool
def dedupe_by_domain(results: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Deduplicates research results by domain, keeping the strongest items per domain.

    Args:
        results (List[Dict[str, Any]]): A list of search result objects where each item contains:
            - url (str): Result URL (used to extract domain).
            - title (str): Page title.
            - content (str): Snippet or summary.
            - score (float): Relevance score.

    Returns:
        List[Dict[str, Any]]: A list of results with at most 'max_per_domain' per domain,
        sorted within each domain by descending 'score' (then title length as tie-breaker).

    Notes:
        If a URL is missing or unparsable, the domain is treated as "unknown".

    Example:
        >>> dedupe_by_domain(search_payload['results'])
        [{'url': 'https://a.com/...', ...}, {'url': 'https://b.org/...', ...}]
    """
    by_domain: Dict[str, List[Dict[str, Any]]] = {}
    for r in results or []:
        domain = urlparse(r.get("url", "")).netloc or "unknown"
        by_domain.setdefault(domain, []).append(r)

    cleaned = []
    max_per_domain = 1

    for _, items in by_domain.items():
        items = sorted(items, key=lambda x: (x.get("score", 0), len(x.get("title", ""))), reverse=True)
        cleaned.extend(items[:max_per_domain])
    return cleaned

In [14]:
researcher_agent = create_react_agent(
    name="researcher_agent",
    prompt=SystemMessage(
        content=(
            "You're a senior marketing researcher. "
            "Given the topic and optionally other sections, "
            "find RECENT, high-signal web information about the topic. "
            "Call web_search at least once with a good query; "
            "then call dedupe_by_domain to remove duplicates. "
            "Return ONLY JSON with key 'research', an array of items "
            "each like {url,title,content,score} (deduped)."
        )
    ),
    model=llm,
    tools=[web_search, dedupe_by_domain],
)

In [15]:
researcher_agent_result = researcher_agent.invoke(
    input={
        "messages": [
            user_trigger_message,
            intake_agent_result["messages"][-1],
        ]
    }
)

In [16]:
for message in researcher_agent_result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I want a blog post about MCP
================================== Ai Message ==================================
Name: intake_agent

{
  "topic": "MCP",
  "title": "MCP: What It Is, Why It Matters, and How To Start",
  "audience": "Practitioners and curious newcomers",
  "sections": [
    "Introduction",
    "Why MCP matters right now",
    "Core concepts in MCP",
    "Practical getting-started guide for MCP",
    "Common pitfalls and best practices",
    "Trends and the future of MCP",
    "Conclusion and next steps"
  ],
  "keywords": [
    "mcp",
    "mcp tutorial",
    "mcp guide",
    "mcp best practices",
    "mcp examples",
    "mcp use cases",
    "mcp vs alternatives",
    "mcp performance",
    "mcp scalability",
    "mcp architecture",
    "mcp tools",
    "mcp pitfalls"
  ]
}
================================== Ai Message ==================================
Name: researcher_agent
Tool Calls:
  web_

## 3. Writer Agent

In [17]:
@tool
def format_references(results: List[Dict[str, Any]]) -> str:
    """
    Formats research results into a Markdown footnotes block.

    Args:
        results (List[Dict[str, Any]]): A list of research items with fields:
            - title (str)
            - url (str)

    Returns:
        str: Markdown string with one footnote per line using the format:
             "[^n]: {title} — {url}"

    Example:
        >>> format_references([{'title':'Doc','url':'https://...'}])
        '[^1]: Doc — https://...'
    """
    lines = []
    for i, r in enumerate(results or [], start=1):
        title = r.get("title", "Untitled")
        url = r.get("url", "")
        lines.append(f"[^{i}]: {title} — {url}")
    return "\n".join(lines)


In [18]:
@tool
def estimate_read_time(markdown_text: str, wpm: int = 225) -> Dict[str, Any]:
    """
    Estimates reading time and word count for a Markdown draft.

    Args:
        markdown_text (str): The full Markdown content to analyze.
        wpm (int, optional): Reading speed in words per minute. Defaults to 225.

    Returns:
        Dict[str, Any]: A dictionary with:
            - word_count (int): Total number of words.
            - minutes (int): Estimated whole minutes to read (minimum of 1).

    Example:
        >>> estimate_read_time("# Title\\n\\nSome text.")
        {'word_count': 3, 'minutes': 1}
    """
    import re
    words = re.findall(r"\b\w+\b", markdown_text or "")
    wc = len(words)
    minutes = max(1, round(wc / max(100, wpm)))
    return {"word_count": wc, "minutes": minutes}

In [19]:
writer_agent = create_react_agent(
    name="writer_agent",
    prompt=SystemMessage(
        content=(
            "You are a seasoned technical blogger. "
            "Write a polished Markdown blog post using title, sections, "
            "and research (array of {url,title,content,score}). "
            "Requirements:\n"
            " - Begin with an H1 (= the title).\n"
            " - Add a short abstract (1-2 sentences), then follow the sections.\n"
            " - Use bullets, code fences, and examples when helpful.\n"
            " - Use footnote-style citations [^1], [^2] inline whenever claims come from research.\n"
            " - Append '## References' followed by footnotes generated via the references tool.\n"
            " - Add an estimated reading time line near the top.\n"
            "Workflow:\n"
            " 1) Draft the post body.\n"
            " 2) Call estimate_read_time on the draft and insert a line like '_Estimated reading time: X min_'.\n"
            " 3) Call format_references with the same research list to create footnotes.\n"
            "Return ONLY JSON with keys: draft_markdown, references_markdown."
        )
    ),
    model=llm,
    tools=[estimate_read_time, format_references],
)

In [20]:
writer_agent_result = writer_agent.invoke(
    input={
        "messages": [
            user_trigger_message,
            intake_agent_result["messages"][-1],
            researcher_agent_result["messages"][-1],
        ]
    }
)

In [21]:
for message in writer_agent_result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I want a blog post about MCP
================================== Ai Message ==================================
Name: intake_agent

{
  "topic": "MCP",
  "title": "MCP: What It Is, Why It Matters, and How To Start",
  "audience": "Practitioners and curious newcomers",
  "sections": [
    "Introduction",
    "Why MCP matters right now",
    "Core concepts in MCP",
    "Practical getting-started guide for MCP",
    "Common pitfalls and best practices",
    "Trends and the future of MCP",
    "Conclusion and next steps"
  ],
  "keywords": [
    "mcp",
    "mcp tutorial",
    "mcp guide",
    "mcp best practices",
    "mcp examples",
    "mcp use cases",
    "mcp vs alternatives",
    "mcp performance",
    "mcp scalability",
    "mcp architecture",
    "mcp tools",
    "mcp pitfalls"
  ]
}
================================== Ai Message ==================================
Name: researcher_agent

{
  "research": [

## 4. Reviewer Agent

In [22]:
@tool
def markdown_lint(md: str) -> Dict[str, Any]:
    """
    Runs a simple Markdown lint pass for structure and readability issues.

    Args:
        md (str): Markdown content to lint.

    Returns:
        Dict[str, Any]: A dictionary with:
            - ok (bool): True if no issues found, else False.
            - issues (List[str]): Human-readable problem descriptions.

    Checks:
        - Ensures the document starts with a single H1 ('# ').
        - Warns if footnote markers are present but definitions are missing.
        - Flags overly long paragraphs (very rough heuristic).

    Example:
        >>> markdown_lint("No title")
        {'ok': False, 'issues': ['Missing H1 title at the very top (use '# Title').']}
    """
    issues = []
    if not md.strip().startswith("# "):
        issues.append("Missing H1 title at the very top (use '# Title').")

    if "[^" in md and "]: " not in md:
        issues.append("Footnote marker used but no footnote definitions present.")

    for idx, para in enumerate([p for p in md.split("\n\n") if p.strip()], start=1):
        if len(para) > 1200:
            issues.append(f"Paragraph {idx} is very long; consider splitting.")

    return {"ok": len(issues) == 0, "issues": issues}

In [23]:
@tool
def seo_score(md: str, keywords: List[str]) -> Dict[str, Any]:
    """
    Computes a naive SEO score (0-100) based on keyword distribution.

    Args:
        md (str): Markdown content to score.
        keywords (List[str]): Target keywords/phrases to check for.

    Returns:
        Dict[str, Any]: A dictionary with:
            - score (int): SEO score (capped at 100).
            - title_hits (int): Count of keywords present in the H1 title.
            - heading_hits (int): Count of keywords found across headings.
            - body_hits (int): Count of keywords found in the full text.
            - keywords_checked (List[str]): Echo of the keywords analyzed.

    Scoring:
        - Title matches weighted 20 points each.
        - Heading matches weighted 6 points each.
        - Body matches weighted 2 points each.

    Example:
        >>> seo_score(md="# LangGraph Guide", keywords=["langgraph"])
        {'score': 28, 'title_hits': 1, 'heading_hits': 0, 'body_hits': 1, 'keywords_checked': ['langgraph']}
    """
    import re
    text = md.lower()
    title = ""
    headings = []
    for line in md.splitlines():
        if line.startswith("# "):
            title = line.lower()
        if line.startswith("#"):
            headings.append(line.lower())

    def contains(k: str, hay: str) -> bool:
        return re.search(rf"\b{re.escape(k)}\b", hay) is not None

    title_hits = sum(1 for k in keywords if contains(k, title))
    heading_hits = sum(1 for k in keywords if any(contains(k, h) for h in headings))
    body_hits = sum(1 for k in keywords if contains(k, text))

    score = min(100, title_hits * 20 + heading_hits * 6 + body_hits * 2)
    return {
        "score": score,
        "title_hits": title_hits,
        "heading_hits": heading_hits,
        "body_hits": body_hits,
        "keywords_checked": keywords[:],
    }

In [24]:
reviewer_agent = create_react_agent(
    name="reviewer_agent",
    prompt=SystemMessage(
        content=(
            "You are a meticulous copy editor. "
            "Input: draft_markdown, keywords, references_markdown. "
            "Tasks:\n"
            " 1) Ensure '## References' exists at the end and merge references_markdown if missing.\n"
            " 2) Call markdown_lint on the draft; then fix issues inline (H1 title, footnotes, long paragraphs).\n"
            " 3) Call seo_score; make small, high-quality edits to improve the score "
            "    without keyword stuffing (preserve voice and clarity).\n"
            "Output: Return ONLY JSON with keys: final_markdown, seo, lint."
        )
    ),
    model=llm,
    tools=[markdown_lint, seo_score],
)

In [25]:
reviewer_agent_result = reviewer_agent.invoke(
    input={
        "messages": [
            user_trigger_message,
            intake_agent_result["messages"][-1],
            researcher_agent_result["messages"][-1],
            writer_agent_result["messages"][-1],
        ]
    }
)

In [26]:
for message in reviewer_agent_result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I want a blog post about MCP
================================== Ai Message ==================================
Name: intake_agent

{
  "topic": "MCP",
  "title": "MCP: What It Is, Why It Matters, and How To Start",
  "audience": "Practitioners and curious newcomers",
  "sections": [
    "Introduction",
    "Why MCP matters right now",
    "Core concepts in MCP",
    "Practical getting-started guide for MCP",
    "Common pitfalls and best practices",
    "Trends and the future of MCP",
    "Conclusion and next steps"
  ],
  "keywords": [
    "mcp",
    "mcp tutorial",
    "mcp guide",
    "mcp best practices",
    "mcp examples",
    "mcp use cases",
    "mcp vs alternatives",
    "mcp performance",
    "mcp scalability",
    "mcp architecture",
    "mcp tools",
    "mcp pitfalls"
  ]
}
================================== Ai Message ==================================
Name: researcher_agent

{
  "research": [

## 5. Publisher Agent

In [27]:
@tool
def publish_blog(body_md: str) -> Dict[str, Any]:
    """
    Publishes the final blog post (mock implementation).

    Args:
        body_md (str): Final Markdown body content to publish.

    Returns:
        Dict[str, Any]: A dictionary indicating publication status:
            - status (str): Always 'published' in this mock implementation.

    Side Effects:
        Prints the blog title and body to stdout, followed by a status line.

    Example:
        >>> publish_blog(title="My Post", body_md="# My Post\\n\\nHello")
        {'status': 'published'}
    """
    print("\n=== PUBLISHING BLOG ===")
    print(body_md)
    print("\n=== STATUS: published ===")
    return {"status": "published"}

In [28]:
publisher_agent = create_react_agent(
    name="publisher_agent",
    prompt=SystemMessage(
        content=(
            "You are a release manager. "
            "Publish the finalized blog post using the provided tool. "
            "Input: title and final_markdown (produced by the reviewer). "
            "Return ONLY JSON with keys: publish_status."
        )
    ),
    model=llm,
    tools=[publish_blog],
)

In [29]:
publisher_agent_result = publisher_agent.invoke(
    input={
        "messages": [
            user_trigger_message,
            intake_agent_result["messages"][-1],
            researcher_agent_result["messages"][-1],
            writer_agent_result["messages"][-1],
            reviewer_agent_result["messages"][-1],
        ]
    }
)


=== PUBLISHING BLOG ===
# MCP: What It Is, Why It Matters, and How To Start

MCP, or Model Control Protocol, is a framework that is gaining traction in various industries for its ability to streamline processes and enhance decision-making. This blog post will explore what MCP is, why it matters, and how you can get started with it.

## Why MCP Matters Right Now

- **Efficiency**: MCP helps organizations improve operational efficiency by automating decision-making processes.
- **Data-Driven Decisions**: With enhanced compliance reports, MCP allows for better data analysis and insights.
- **Future Trends**: The adoption of MCP is predicted to rise in smart manufacturing, leading to increased productivity and improved decision-making.

## Core Concepts in MCP

- **Transparency**: Ensuring that AI decision-making processes are explainable.
- **Compliance**: Understanding the compliance landscape and how MCP can help navigate it.
- **Security**: Identifying key risks and implementing best 

In [30]:
for message in publisher_agent_result["messages"]:
    message.pretty_print()

================================ Human Message =================================

I want a blog post about MCP
================================== Ai Message ==================================
Name: intake_agent

{
  "topic": "MCP",
  "title": "MCP: What It Is, Why It Matters, and How To Start",
  "audience": "Practitioners and curious newcomers",
  "sections": [
    "Introduction",
    "Why MCP matters right now",
    "Core concepts in MCP",
    "Practical getting-started guide for MCP",
    "Common pitfalls and best practices",
    "Trends and the future of MCP",
    "Conclusion and next steps"
  ],
  "keywords": [
    "mcp",
    "mcp tutorial",
    "mcp guide",
    "mcp best practices",
    "mcp examples",
    "mcp use cases",
    "mcp vs alternatives",
    "mcp performance",
    "mcp scalability",
    "mcp architecture",
    "mcp tools",
    "mcp pitfalls"
  ]
}
================================== Ai Message ==================================
Name: researcher_agent

{
  "research": [